<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf


    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-07-11 11:46:06
-------------------
qualified stocks: 96
with latest results: 31
still star stocks: 20
-------------------
Initial Investment:  1.17 C
CY Investment:  1.44 C
Reserve:  9.00 K
Current:  1.36 C
-------------------
Today PnL: -77.42 K (-0.56%)
Current PnL: -13.73 L (-9.57%)
CY Booked + Current PnL: -6.77 L (-4.72%)
-------------------
Total profit:  4.72 L
Total loss:  -18.46 L
-------------------
Total Booked + Current PnL: 20.21 L (17.34%)
Total Booked PnL: 33.94 L (29.12%)
Curr Year Booked PnL: 6.96 L (5.1%)
Prev Year Booked PnL: 26.98 L (23.15%)
Est FTT:  2.09 C
Est FTT PnL: 72.28 L (52.99%)
Deployed:  1.17 C
Current:  1.36 C
CAGR/XIRR %: 10.54%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
92,VIPIND,488.80,489.00,-226.13,H-SC,95.83,90948.0,-4368.0,4411.0,3.92,-4.58,4.85,0.04,157.0,-0.99,0.67,79.38,OX40N,NTT,MISC
87,UJJIVANSFB,52.77,53.00,48.19,M-SC,23.96,128763.0,-13716.0,14331.0,0.32,-9.63,11.13,0.44,247.0,-0.96,0.94,51.98,OX40N,NTT,BANKS
39,HINDALCO,651.95,761.55,-22.38,H-LC,76.04,107416.0,3104.0,14437.0,-0.55,2.98,13.44,16.81,27.0,0.22,0.79,19.50,X5K,ATH,METALS
71,SBIN,760.30,863.00,-11.51,M-LC,51.04,213961.0,12482.0,14742.0,-0.07,6.19,6.89,13.51,77.0,0.85,1.57,19.57,XY25,NTT,BANKS
52,JPPOWER,18.73,26.20,-9.01,L-SC,100.00,183039.0,41197.0,15375.0,4.95,29.04,8.40,39.88,262.0,2.68,1.34,85.50,XY24,NTT,POWER


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
38,HEROMOTOCO,4311.81,6039.03,-7.14,H-MC,32.29,149552.0,-1361.0,61810.0,-1.12,-0.90,41.33,40.06,103.0,-0.02,1.10,22.32,AR,ATH,AUTO
60,NATIONALUM,189.63,247.44,-50.15,H-MC,62.50,100976.0,472.0,30172.0,0.83,0.47,29.88,30.49,97.0,0.02,0.74,33.03,MH,ATH,METALS


In [11]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
52,JPPOWER,18.73,26.2,-9.01,L-SC,100.00,183039.0,41197.0,15375.0,4.95,29.04,8.40,39.88,262.0,2.68,1.34,85.50,XY24,NTT,POWER
40,HINDUNILVR,2413.81,2723.0,-9.90,X-LC,89.58,282229.0,11882.0,22748.0,4.63,4.40,8.06,12.81,5.0,0.52,2.07,18.03,XY25,NTT,FMCG
92,VIPIND,488.80,489.0,-226.13,H-SC,95.83,90948.0,-4368.0,4411.0,3.92,-4.58,4.85,0.04,157.0,-0.99,0.67,79.38,OX40N,NTT,MISC
56,LAOPALA,369.40,464.0,125.48,H-SC,35.42,71567.0,-29279.0,55107.0,2.82,-29.03,77.00,25.61,136.0,-0.53,0.53,29.07,AR,NTT,CERAMICS
63,QUESS,296.83,986.0,-31.68,M-SC,41.67,67890.0,2884.0,148041.0,1.49,4.44,218.06,232.18,193.0,0.02,0.50,3.96,XY24,NTT,MISC


In [12]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
20,CAMS,3643.00,5250.99,0.76,H-SC,21.88,113148.0,11144.0,33877.0,-3.51,10.93,29.94,44.14,126.0,0.33,0.83,28.94,X40N,ATH,MISC
69,SAMMAANCAP,170.35,326.00,-165.91,M-SC,40.62,76554.0,-25656.0,119049.0,-3.34,-25.10,155.51,91.37,212.0,-0.22,0.56,24.27,XY25,NTT,FINANCE
21,CERA,8421.60,8422.00,-16.47,X-SC,22.92,86229.0,-23252.0,23256.0,-2.84,-21.24,26.97,0.00,69.0,-1.00,0.63,31.75,OX40N,NTT,CERAMICS
83,TCS,3794.03,4998.00,-21.27,X-LC,14.58,256612.0,-39322.0,133233.0,-2.72,-13.29,51.92,31.73,1.0,-0.30,1.88,2.71,X40,BTT,IT
65,RELAXO,902.64,1176.00,-28.54,H-SC,96.88,64640.0,-50898.0,85887.0,-2.67,-44.05,132.87,30.28,131.0,-0.59,0.47,26.28,X40N,NTT,FOOTWEAR


In [13]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
92,VIPIND,488.80,489.0,-226.13,H-SC,95.83,90948.0,-4368.0,4411.0,3.92,-4.58,4.85,0.04,157.0,-0.99,0.67,79.38,OX40N,NTT,MISC
87,UJJIVANSFB,52.77,53.0,48.19,M-SC,23.96,128763.0,-13716.0,14331.0,0.32,-9.63,11.13,0.44,247.0,-0.96,0.94,51.98,OX40N,NTT,BANKS
79,SYMPHONY,1306.42,1306.0,-28.99,M-SC,34.38,151253.0,-19888.0,19829.0,-0.47,-11.62,13.11,-0.03,191.0,-1.00,1.11,8.71,OX40N,NTT,DURABLES
46,INDIGOPNTS,1407.73,1408.0,127.12,H-SC,87.50,148825.0,-25734.0,25762.0,0.00,-14.74,17.31,0.02,133.0,-1.00,1.09,29.05,OX40N,NTT,PAINTS
53,KANSAINER,299.63,340.0,-68.02,H-SC,47.92,222975.0,-46692.0,83036.0,-0.54,-17.31,37.24,13.47,137.0,-0.56,1.64,11.60,XY24,NTT,PAINTS


In [14]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,TCS,3794.03,4998.00,-21.27,X-LC,14.58,256612.0,-39322.0,133233.0,-2.72,-13.29,51.92,31.73,1.0,-0.30,1.88,2.71,X40,BTT,IT
48,INFY,1461.46,2275.00,-11.34,X-LC,36.46,284789.0,23188.0,122431.0,-1.53,8.86,42.99,55.67,2.0,0.19,2.09,15.51,X40,BTT,IT
40,HINDUNILVR,2413.81,2723.00,-9.90,X-LC,89.58,282229.0,11882.0,22748.0,4.63,4.40,8.06,12.81,5.0,0.52,2.07,18.03,XY25,NTT,FMCG
17,BRITANNIA,4983.33,6446.05,17.04,X-LC,78.12,268272.0,39039.0,28249.0,-0.78,17.03,10.53,29.35,10.0,1.38,1.97,27.47,XY25,ATH,FMCG
74,SIEMENS,4406.90,7969.85,-7.62,X-LC,10.42,78040.0,-32132.0,121204.0,-2.19,-29.17,155.31,80.85,11.0,-0.27,0.57,14.94,AR,ATH,ELECTRICAL


In [15]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,ENRIN,1377.95,3140.3,NaN,NaN,98.96,78375.0,43926.0,133.0,-0.17,127.51,0.17,127.90,274.0,330.27,0.58,19.85,AR,ATH,ELECTRICAL
52,JPPOWER,18.73,26.2,-9.01,L-SC,100.00,183039.0,41197.0,15375.0,4.95,29.04,8.40,39.88,262.0,2.68,1.34,85.50,XY24,NTT,POWER
59,MEDANTA,1064.65,1486.0,17.74,X-SC,92.71,251170.0,45693.0,35616.0,-0.03,22.24,14.18,39.58,65.0,1.28,1.84,34.60,XY24,NTT,HEALTHCARE


In [16]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
68,SAIL,130.64,228.00,93.57,M-MC,69.79,232401.0,7439.0,160217.0,0.13,3.31,68.94,74.53,189.0,0.05,1.71,34.93,XY24,BTT,STEEL
54,KPIGREEN,497.21,732.01,28.70,H-SC,46.88,131645.0,6348.0,52816.0,-2.20,5.07,40.12,47.22,143.0,0.12,0.97,64.70,MH,ATH,POWER
20,CAMS,3643.00,5250.99,0.76,H-SC,21.88,113148.0,11144.0,33877.0,-3.51,10.93,29.94,44.14,126.0,0.33,0.83,28.94,X40N,ATH,MISC
15,BLUESTARCO,1646.70,2337.55,-4.35,H-SC,93.75,179500.0,14830.0,54263.0,-0.09,9.01,30.23,41.95,124.0,0.27,1.32,17.14,X40N,ATH,AC
88,UNIONBANK,123.87,163.00,-6.01,M-LC,12.50,163432.0,22592.0,21900.0,-0.08,16.04,13.40,31.59,88.0,1.03,1.20,41.97,XY24,NTT,BANKS


In [17]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,ENRIN,1377.95,3140.30,NaN,NaN,98.96,78375.0,43926.0,133.0,-0.17,127.51,0.17,127.90,274.0,330.27,0.58,19.85,AR,ATH,ELECTRICAL
94,WHIRLPOOL,1167.49,2270.00,-40.20,M-SC,55.21,197303.0,25682.0,136396.0,1.05,14.96,69.13,94.43,201.0,0.19,1.45,45.62,XR,NTT,DURABLES
54,KPIGREEN,497.21,732.01,28.70,H-SC,46.88,131645.0,6348.0,52816.0,-2.20,5.07,40.12,47.22,143.0,0.12,0.97,64.70,MH,ATH,POWER
45,INDIAMART,2327.09,4911.36,-47.33,H-SC,81.25,137281.0,13945.0,123018.0,0.15,11.31,89.61,111.05,123.0,0.11,1.01,34.51,AR,ATH,MISC
95,WIPRO,243.46,420.00,-6.49,M-LC,59.38,160239.0,9294.0,100165.0,-2.49,6.16,62.51,72.51,71.0,0.09,1.18,10.38,XR,NTT,IT


In [18]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
16,BLUSPRING,226.45,86.36,NaN,NaN,67.71,18672.0,-30921.0,241.0,-1.27,-62.35,1.29,-61.86,270.0,-128.30,0.14,25.22,XY24,ATH,MISC
24,COFFEEDAY,59.14,80.00,-55.73,L-SC,9.38,66374.0,-47175.0,87222.0,-0.89,-41.55,131.41,35.27,265.0,-0.54,0.49,62.45,XR,NTT,HOTELS
32,ENRIN,1377.95,3140.30,NaN,NaN,98.96,78375.0,43926.0,133.0,-0.17,127.51,0.17,127.90,274.0,330.27,0.58,19.85,AR,ATH,ELECTRICAL
6,ASIANTILES,75.41,137.00,7211.11,M-SC,82.29,80238.0,-13572.0,90188.0,0.11,-14.47,112.40,81.67,245.0,-0.15,0.59,55.38,XR,NTT,CERAMICS
92,VIPIND,488.80,489.00,-226.13,H-SC,95.83,90948.0,-4368.0,4411.0,3.92,-4.58,4.85,0.04,157.0,-0.99,0.67,79.38,OX40N,NTT,MISC


In [19]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
52,JPPOWER,18.73,26.20,-9.01,L-SC,100.00,183039.0,41197.0,15375.0,4.95,29.04,8.40,39.88,262.0,2.68,1.34,85.50,XY24,NTT,POWER
32,ENRIN,1377.95,3140.30,NaN,NaN,98.96,78375.0,43926.0,133.0,-0.17,127.51,0.17,127.90,274.0,330.27,0.58,19.85,AR,ATH,ELECTRICAL
3,ALKYLAMINE,2347.98,4546.37,4.74,X-SC,97.92,97429.0,-3534.0,98062.0,-1.91,-3.50,100.65,93.63,67.0,-0.04,0.71,45.12,SR,ATH,CHEMICALS
65,RELAXO,902.64,1176.00,-28.54,H-SC,96.88,64640.0,-50898.0,85887.0,-2.67,-44.05,132.87,30.28,131.0,-0.59,0.47,26.28,X40N,NTT,FOOTWEAR
92,VIPIND,488.80,489.00,-226.13,H-SC,95.83,90948.0,-4368.0,4411.0,3.92,-4.58,4.85,0.04,157.0,-0.99,0.67,79.38,OX40N,NTT,MISC


In [20]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.03
1,25,41.98
2,50,70.67


In [21]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.73
LC    33.64
MC    21.58
Name: CurrAlloc%, dtype: float64

In [22]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     31.20
X40      12.79
XY25     12.16
X40N     11.87
XR        9.94
OX40N     8.20
AR        8.17
MH        1.71
X5K       1.42
X200      1.38
SR        1.19
Name: CurrAlloc%, dtype: float64

In [23]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    20.54
M-SC    15.91
H-LC    13.96
X-LC    12.85
H-MC     9.59
X-MC     7.57
M-LC     5.79
X-SC     4.55
M-MC     4.06
L-SC     2.73
L-LC     1.04
L-MC     0.36
Name: CurrAlloc%, dtype: float64

In [24]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.36,0.13,33.34
IT,11.88,-12.27,67.45
FINANCE,8.48,-12.78,53.70
BANKS,7.64,-9.45,54.11
MISC,5.89,-20.00,76.29
PAINTS,5.66,-13.17,35.48
ELECTRICAL,5.43,0.43,46.63
HEALTHCARE,4.74,0.48,28.59
AUTO,4.30,-15.53,66.34


In [25]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2640741.0,26
XR,1105431.0,13
AR,1039275.0,10
X40,712576.0,10
X40N,544843.0,11
XY25,461422.0,8
OX40N,311511.0,11
SR,181316.0,2
MH,82988.0,2


In [26]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,1989146.0,20
M-SC,1777192.0,21
H-LC,638226.0,12
X-LC,557800.0,8
H-MC,526395.0,8
X-MC,420747.0,6
M-MC,416488.0,3
X-SC,297730.0,5
M-LC,272922.0,4


In [27]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      899803.0      6
M-SC       XY24      807869.0      7
H-SC       AR        525203.0      3
M-SC       XR        340799.0      4
M-MC       XY24      299355.0      2
H-MC       XY24      297624.0      3
X-LC       X40       292484.0      3
H-SC       XR        219656.0      3
           X40N      207918.0      4
H-LC       X40N      203667.0      4
M-SC       AR        187913.0      2
           OX40N     167630.0      5
H-LC       X40       160796.0      3
X-MC       X40       159986.0      2
M-LC       XY24      158015.0      2
L-SC       XR        150979.0      2
X-SC       XR        122891.0      1
X-LC       AR        121204.0      1
M-SC       XY25      119049.0      1
M-MC       XR        117133.0      1
X-MC       XY25      106440.0      1
M-LC       XR        100165.0      1
X-SC       SR         98062.0      1
X-LC       XY25       89837.0      3
H-LC       AR         84340.0      1
H-SC       OX40N      83750.0      3
M-SC       SR         83254.0      1
H-MC       XY25       83199.0      1
X-MC       XY24       81326.0      1
H-LC       X5K        75050.0      2
           X200       72488.0      1
M-SC       X40        70678.0      1
H-MC       AR         61810.0      1
L-SC       AR         58672.0      1
X-LC       X40N       54275.0      1
X-MC       X40N       54025.0      1
L-MC       XR         53808.0      1
H-SC       MH         52816.0      1
L-LC       XY25       48155.0      1
H-LC       XY24       41885.0      1
X-SC       OX40N      41161.0      2
           XY24       35616.0      1
H-MC       MH         30172.0      1
           X40        28632.0      1
           X40N       24958.0      1
X-MC       OX40N      18970.0      1
L-SC       XY24       15375.0      1
M-LC       XY25       14742.0      1

In [28]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 32.0 seconds
